In [7]:
import spacy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import nltk
import re,string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("C:\\Users\\Admin\\Downloads\\twitter_data.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

(24783, 7)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [8]:
nlp = spacy.load("en_core_web_lg")

In [9]:
df["labels"] = df['class'].map({0: "Hate Speech", 1: "Offensive Speech", 2: "No Hate and Offensive Speech"})
df = df[["tweet", "labels"]]

In [4]:
df["labels"] = df['class'].map({0: "No Hate and Offensive Speech", 1: "Offensive Speech"})
df = df[["tweet", "labels"]]

KeyError: 'class'

In [10]:
def preprocess(text):
        text = str(text)
        test_list = text.split()
        text = ""
        for word in test_list:
            if word.endswith("ing"):
                word=word[:-3]
            text+=word
            text+=" "
        doc = nlp(text)
        filtered_tokens = []
        
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            filtered_tokens.append(token.lemma_)
            
        return " ".join(filtered_tokens)

In [11]:
df["tweet_new"] = df.tweet.apply(preprocess)

In [12]:
df.head()

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,RT @mayasolovely woman complain clean house am...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,RT @mleew17 boy dats cold tyga dwn bad cuffin ...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,RT @urkindofbrand Dawg RT @80sbaby4life fuck b...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,RT @C_G_Anderson @viva_based look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,RT @ShenikaRoberts shit hear true faker bitch ...


In [13]:
def clean(text):
    text = str (text). lower()
    text = re. sub('[.?]', '', text)
    text = re. sub('https?://\S+|www.\S+', '', text)
    text = re. sub('<.?>+', '', text)
    text = re. sub('[%s]' % re. escape(string. punctuation), '', text)
    text = re. sub('\n', '', text)
    text = re. sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ". join(text)
    text = [stemmer. stem(word) for word in text. split(' ')]
    text=" ". join(text)
    return text

In [14]:
df["tweet_new"] = df.tweet_new.apply(clean)

In [15]:
df['vector'] = df['tweet_new'].apply(lambda text: nlp(text).vector)  

In [16]:
#Add the new column which gives a unique number to each of these labels 
df['label_num'] = df['labels'].map({'No Hate and Offensive Speech' : 0, 'Offensive Speech': 1, 'Hate Speech':2})

#check the results with top 5 rows
df.head(5)

,tweet,labels,tweet_new,vector,label_num
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,rt mayasolov woman complain clean hous amp man...,"[-0.57211995, 0.6595566, -1.6417766, -0.351363...",0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,rt mlee boy dat cold tyga dwn bad cuffin dat h...,"[-0.36383075, -0.010934628, -0.44984773, 0.124...",1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,rt urkindofbrand dawg rt babif fuck bitch star...,"[0.3710009, 0.51078933, -1.34232, 0.4083718, -...",1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,rt cganderson vivabas look like tranni,"[-0.033501666, 0.59515333, -0.56129, -0.349748...",1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,rt shenikarobert shit hear true faker bitch te...,"[0.54850054, 0.15953834, -0.31898752, -0.46431...",1


In [17]:
#Add the new column which gives a unique number to each of these labels 
df['label_num'] = df['labels'].map({'No Hate and Offensive Speech' : 0, 'Offensive Speech': 1,})

#check the results with top 5 rows
df.head(5)

,tweet,labels,tweet_new,vector,label_num
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,rt mayasolov woman complain clean hous amp man...,"[-0.57211995, 0.6595566, -1.6417766, -0.351363...",0.0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,rt mlee boy dat cold tyga dwn bad cuffin dat h...,"[-0.36383075, -0.010934628, -0.44984773, 0.124...",1.0
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,rt urkindofbrand dawg rt babif fuck bitch star...,"[0.3710009, 0.51078933, -1.34232, 0.4083718, -...",1.0
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,rt cganderson vivabas look like tranni,"[-0.033501666, 0.59515333, -0.56129, -0.349748...",1.0
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,rt shenikarobert shit hear true faker bitch te...,"[0.54850054, 0.15953834, -0.31898752, -0.46431...",1.0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.label_num, test_size=0.2, random_state=2022)

In [19]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

def classify_text(input_text):
    preprocessed_text = preprocess(input_text)
    cleaned_text = clean(preprocessed_text)
    vector = nlp(cleaned_text).vector
    scaled_vector = scaler.transform([vector])
    prediction = clf.predict(scaled_vector)
    label_mapping = {0: 'No Hate and Offensive Speech', 1: 'Offensive Speech', 2: 'Hate Speech'}
    predicted_label = label_mapping.get(prediction[0], 'Unknown')
    return predicted_label

# Enter your string for classification
input_string = "hi"
result = classify_text(input_string)
print(f"Predicted Label: {result}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [21]:
from  sklearn.neighbors import KNeighborsClassifier

#1. creating a KNN model object
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)

#4. print the classfication report
print(classification_report(y_test, y_pred))

def classify_text(input_text):
    preprocessed_text = preprocess(input_text)
    cleaned_text = clean(preprocessed_text)
    vector = nlp(cleaned_text).vector
    scaled_vector = scaler.transform([vector])
    prediction = clf.predict(scaled_vector)
    label_mapping = {0: 'No Hate and Offensive Speech', 1: 'Offensive Speech', 2: 'Hate Speech'}
    predicted_label = label_mapping.get(prediction[0], 'Unknown')
    return predicted_label

# Enter your string for classification
input_string = "bye"
result = classify_text(input_string)
print(f"Predicted Label: {result}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


svc = SVC()
svc.fit(scaled_train_embed, y_train)

y_pred = svc.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

def classify_text(input_text):
    preprocessed_text = preprocess(input_text)
    cleaned_text = clean(preprocessed_text)
    vector = nlp(cleaned_text).vector
    scaled_vector = scaler.transform([vector])
    prediction = svc.predict(scaled_vector)
    label_mapping = {0: 'No Hate and Offensive Speech', 1: 'Offensive Speech', 2: 'Hate Speech'}
    predicted_label = label_mapping.get(prediction[0], 'Unknown')
    return predicted_label

# Enter your string for classification
input_string = "bye"
result = classify_text(input_string)
print(f"Predicted Label: {result}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


model = RandomForestClassifier()
model.fit(scaled_train_embed, y_train)

y_pred = model.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

def classify_text(input_text):
    preprocessed_text = preprocess(input_text)
    cleaned_text = clean(preprocessed_text)
    vector = nlp(cleaned_text).vector
    scaled_vector = scaler.transform([vector])
    prediction = model.predict(scaled_vector)
    label_mapping = {0: 'No Hate and Offensive Speech', 1: 'Offensive Speech', 2: 'Hate Speech'}
    predicted_label = label_mapping.get(prediction[0], 'Unknown')
    return predicted_label

# Enter your string for classification
input_string = "bye"
result = classify_text(input_string)
print(f"Predicted Label: {result}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


model = MultinomialNB()
model.fit(scaled_train_embed, y_train)

y_pred = model.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

def classify_text(input_text):
    preprocessed_text = preprocess(input_text)
    cleaned_text = clean(preprocessed_text)
    vector = nlp(cleaned_text).vector
    scaled_vector = scaler.transform([vector])
    prediction = model.predict(scaled_vector)
    label_mapping = {0: 'No Hate and Offensive Speech', 1: 'Offensive Speech', 2: 'Hate Speech'}
    predicted_label = label_mapping.get(prediction[0], 'Unknown')
    return predicted_label

# Enter your string for classification
input_string = "bye"
result = classify_text(input_string)
print(f"Predicted Label: {result}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [26]:
from sklearn.ensemble import VotingClassifier

# Initialize individual models
dt_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
svm_model = SVC(kernel='linear', probability=True)  # Enable probability estimation for SVM

# Define the ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('decision_tree', dt_model),
    ('random_forest', rf_model),
    ('svm', svm_model)
], voting='soft')  # Use soft voting for probability averaging

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

# Training the ensemble model
ensemble_model.fit(scaled_train_embed, y_train)

# Testing the ensemble model
y_pred_ensemble = ensemble_model.predict(scaled_test_embed)

# Accuracy Score of ensemble model
print("Ensemble Accuracy Score:", accuracy_score(y_test, y_pred_ensemble))

NameError: name 'DecisionTreeClassifier' is not defined

In [ ]:
# Get input from the user
user_input = input("Enter the text you want to classify: ")

# Preprocess the user input
def classify_text(input_text):
    preprocessed_text = preprocess(input_text)
    cleaned_text = clean(preprocessed_text)
    vector = nlp(cleaned_text).vector
    scaled_vector = scaler.transform([vector])
    prediction = ensemble_model.predict(scaled_vector)
    label_mapping = {0: 'No Hate and Offensive Speech', 1: 'Offensive Speech', 2: 'Hate Speech'}
    predicted_label = label_mapping.get(prediction[0], 'Unknown')
    return predicted_label

# Enter your string for classification
input_string = "bye"
result = classify_text(input_string)